In [25]:
# import sys
# sys.path.append("/home/baris/repos/rlvr")

# from scripts.musique import evaluate

In [26]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [27]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1756107462035, experiment_id='5', last_update_time=1756107462035, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [28]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-7B-Instruct']
Using model: Qwen/Qwen2.5-7B-Instruct


In [29]:
datasets_str = "bdsaglam/musique-mini,answerable,validation[:8]"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.1
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation[:8] (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.1
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 8/8 [00:00<?, ? examples/s]

2025-08-25 16:19:47 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 8 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 8 rollouts: 100%|██████████| 8/8 [00:00<00:00, 694.74it/s]


[Trace(trace_id=tr-1f48d7f4e0c7370bebb4ae6ab3fcdfcb), Trace(trace_id=tr-0bfc9d815b6c874775e7706ff98268fd), Trace(trace_id=tr-388d321f37e965deeeaeb25d584fe757), Trace(trace_id=tr-d4d7d5e586af04c78ec804ae83b2afe2), Trace(trace_id=tr-940fd6f4291a7dd5a5ea9e843c3f9267), Trace(trace_id=tr-9c1dbb59aa2105de93715da5dc85b7db), Trace(trace_id=tr-53e7c4256cfa4e073ee70530bb18d60a), Trace(trace_id=tr-012e91ae588581ccfa8cb15a60489ac5), Trace(trace_id=tr-1e10ec952a3a4601e522bd38f7311cf0), Trace(trace_id=tr-5bb2d9bd07fcdf0839ed397ff1607cc6)]

In [30]:
len(results.prompt), len(results.completion), len(results.answer), len(results.state), len(results.info), len(results.task), len(results.reward)

(8, 8, 8, 8, 8, 8, 8)

In [31]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [32]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'content': "Answer the question based on the information provided by tools.\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve relevant documents\n3. Continue until you have found the answer through multi-hop reasoning\n4. In the **last** step:\n    - Reflect on your previous steps inside <think> tags\n    - Cite the documents you used inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n    - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking here]\n</think> \n<cite>\n[IDs of the documents that back your answer]\n</cite>\n<answer>\n[your final answer in a few words]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue searching until you find all relevant information to answer the question.", 'role': 'system'}, {'content': 'Who\'s the sibling of the player who won

/tmp/ipykernel_1757079/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [33]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'info', 'task', 'reward', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 8
})

In [34]:
ds.to_json("../outputs/sample.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

250629

In [35]:
df = ds.to_pandas()

In [36]:
metric_cols = [
    'exact_match_reward',
    'f1_reward',
    'retrieval_recall_reward',
    'citation_reward',
    'format_reward',
    'combined_reward',
]
df[metric_cols].describe()

,exact_match_reward,f1_reward,retrieval_recall_reward,citation_reward,format_reward,combined_reward
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.125000,0.301171,0.875000,0.687500,0.833333,0.347953
std,0.353553,0.334924,0.231455,0.258775,0.178174,0.100150
min,0.000000,0.000000,0.500000,0.500000,0.666667,0.200000
25%,0.000000,0.055556,0.875000,0.500000,0.666667,0.286389
50%,0.000000,0.217647,1.000000,0.500000,0.833333,0.348611
75%,0.000000,0.425000,1.000000,1.000000,1.000000,0.399804
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.516667
